# Global Earth Surface Temperature Analysis and Forecast

## Overview

## Data

The data source of the temperature time series used for the modeling is Kaggle <sup>[1](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)</sup>. The data contains global monthly averages of land and ocean temparatures and temperature uncertainty from 1750 to 2015. There are also country and city wise breakups of the monthly temperature means. <br>
The source of the historic CO2 data is Nasa's Global Climate Change website <sup>[2](https://climate.nasa.gov/vital-signs/carbon-dioxide/)</sup>. This data from March 1958 through April 1974 have been originally obtained from the Scripps website <sup>[3](scrippsco2.ucsd.edu)</sup>.

## Exploration and Analysis

Looking at the maximum and minimum temperatures over the last few years, we see the same trend. Both the average maximum and minimum land temperatures have been slowly but surely rising and have increased by more than a degree in the last 50 years.

<img src='./images/max-min-temp.png' >

This rise is evident not only in the land temperatures, but also in the ocean temperatures. Also, this rise clearly matches the increasing trend of CO2 in the atmosphere. CO2 is a greenhouse gas that traps heat and directly contributes to global warming. It's increase can be attributed to deforestation and fossil fuel usage.

<img src='./images/average-temp.png' >

Here is a visual of the temperature change every decade by country between 1880 and 2010. The rise in global temperatures seems to have really accelerated after 1970 due to increased industrialization and urbanization.

<img src='./images/movie.gif' >

## Forecast

In most cases, a time series has 3 components embedded in the data - trend, seasonality and noise. <br>
<p><b>Trend</b> reflects the long-term progression of the series (secular variation). A trend exists when there is a persistent increasing or decreasing direction in the data. The trend component can be linear or non-linear.
<p><b>Seasonality</b> or a seasonal pattern exists when a time series is influenced by seasonal factors. Seasonality occurs over a fixed and known period (e.g., the quarter of the year, the month, or day of the week).
<p><b>Noise</b> or irregular component describes random, irregular influences. It represents the residuals or remainder of the time series after the other components have been removed.   

Considering an additive decomposition, our time series can be broken down as: y<sub>t</sub> = S<sub>t</sub> + T<sub>t</sub> + R<sub>t</sub>, where y<sub>t</sub> is the data,  S<sub>t</sub> is the seasonal component, T<sub>t</sub> is the trend-cycle component, and R<sub>t</sub> is the remainder component, all at period *t*
    
When we decompose our data, we can see a general upward trend with some minor dips. There is very clear seasonality in the data, with 12 data points per seasonal cycle, since the data set has monthly data points.

<img src='./images/decomposition.png' height="50%">

Next, we will check the stationarity of our time series. A time series is "stationary" if all of its statistical properties—mean, variance, autocorrelations, etc. are constant in time and do not depend on the time at which the series is observed <sup>[5](http://people.duke.edu/~rnau/411home.htm)</sup>. So, any series with trend and seasonality components is not stationary. Since, we just saw that our data has both these components, it is unlikely that our time series is stationary. 
Let's verify this by running the augmented Dicky-Fueller test.

One way to make a non-stationary time series stationary is to compute the differences between consecutive observations. This is known as <b>differencing</b>. Here is the result of first order differencing on our data.

Let's run the test again on the differenced data. We see that the p-value is insignificant and we can reject the null hypothesis that a unit root (a characteristic that makes the time series non-stationary) is present in this time series sample. This means the differenced time series is stationary.

Lag plot is a type of scatter plot where observations are plotted with a lagged version of themselves. A lag plot can give us clues about model suitability based on the underlying data. Our data shows a positive linear trend in the lag plot, which is an 
indication of autocorrelation. However, the points are not concentrated only on the diagonal line, so the autocorrelation is not very strong.

<img src='./images/lag-plot.png' >

In the next few steps, we will train and fit a model, then do an in-sample prediction to test how the model performs. If the model performance is satisfoctary, we will do an out of sample forecast.

We will use a seasonal ARIMA (SARIMA) model for our time series forecasting.

A non-seasonal ARIMA model can be summarized by three numbers. This is called an ARIMA(p,d,q) model, where
- p = the number of autoregressive terms
- d = the number of nonseasonal differences
- q = the number of moving-average terms

A seasonal ARIMA model is formed by including additional seasonal terms in the above ARIMA model.
- P = the number of seasonal autoregressive terms
- D = the number of seasonal differences
- Q = the number of seasonal moving-average terms
- m = number of observations per year.


To fit the SARIMA model, the AR and MA terms need to be determined to correct any autocorrelation that remains in the differenced series. One way to tentatively identify them, is to use autocorrelation function (ACF) and partial autocorrelation function (PACF) plots of the differenced series.
Autocorrelation measures the linear relationship between lagged values of a time series. An ACF plot is a bar chart of the coefficients of correlation between a time series and lags of itself.
A partial autocorrelation is the amount of correlation between a variable and a lag of itself that is not explained by correlations at all lower-order-lags.

Here are the ACF and PACF plots of the original and differenced series:
<img src='./images/correlogram.png' >
As a rule of thumb, the lag at which the PACF crosses the upper confidence interval is the indicated number of AR terms and the lag at which the ACF crosses the upper confidence interval is the indicated number of MA terms.
From the figure, it seems like, p=1, d=0 and q=3.

To verify this finding, we will use the auto_arima function from the pmdarima library <sup>[6](http://alkaline-ml.com/pmdarima/index.html)</sup> to find the orders. This function tests models crested using several combinations of p,d and q values till it finds the model with the lowest AIC. The Akaike Information Critera or AIC is a widely used estimator to measure the ampount of error in a statistical model.

The auto_arima function recommends the terms p=0, d=0, q=3, P=2, D=0, Q=2 and m=12. So, we will use these terms to build our model.

Let's do a 90/10 split of our data into training and testing sets. The training set will be used to build the model and the testing set will be used to test the model. 
After the split, the training set contains 1792 samples and the testing set has 199 samples. We then use the training set to build a SARIMA(0, 0, 3)x(2, 0, 2, 12). More detailed results below:

<img src='./images/pred_results.PNG' >

After we fit the model, we make an in-sample prediction of monthly temperatures from 1999-06-01 to 2015-12-01. After the prediction, the predicted values are scaled back to the original level. Then the actual and predicted values are compared by superimposing the predicted temperatures on top of the actual temperatures for this time period. In the plot, the predictions seem to be tracking the actuals quite closely.
The root mean square, which is the root of the difference between the actual and predicted values, is 0.3551. This is also in the acceptable range.

<img src='./images/prediction.png' >

### Out of sample forecasting

The next step is to forecast future temperatures. But, before that we need to run the auto_arima function on the entire dataset, to get the orders for the SARIMA model that will be used to build the forecasting model.

The lowest AIC is observed for SARIMA(0,1,1)x(2, 0, 1, 12). A forecasting model is fit using these orders. Here is the summary of the SARIMA results.

<img src='./images/forecast_results.PNG' >

The model is then used to make a forecast for the next 8 years (96 months). The first 10 forecasted values look like this:

Let's plot the Actual temperatures along with the predicted and forecasted values to see what they it looks like side by side.

<img src='./images/actual-prediction-forecast.png' >

### Results

The forecasted values in the plot indicate that the temperatures will not only continue to rise, but the rate of temperature increase will be significantly accelerated. This finding is broadly consistent not only with that of NASA and Climatic Research Unit, but most climate researchers have predicted a faster rate of warming in the coming years.

## Conclusion

As global temperatures continue to rise, we will soon see parts of the world slowly becoming inhabitable. This will also cause ice sheets to melt and sea levels to rise. If this continues, the consequences will be deadly - many animal species will become extinct and massive amounts of habitats will be lost due to natural calamities. 
Let's hope we rally together to move towards a more sustainable lifestyle so that this dangerous trend is delayed and then reversed.

## References

- https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data
- https://climate.nasa.gov/vital-signs/carbon-dioxide/
- https://scrippsco2.ucsd.edu
- https://otexts.com/fpp2/
- http://people.duke.edu/~rnau/411home.htm
- http://alkaline-ml.com/pmdarima/index.html
- https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3
- https://plotly.com/python/reference/
- https://pypi.org/project/pmdarima/
- https://pandas.pydata.org/pandas-docs/stable/reference/index.html
- https://www.statsmodels.org/stable/index.html
